<a href="https://colab.research.google.com/github/sgr1118/Sentence-Embedding-Using-Korean-Corpora/blob/main/03_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 전처리
- 3장에서는 임베딩 학습을 위한 한국어 데이터 전처리 과정을 다룬다. 웹 문서나 json 같은 형태의 파일을 텍스트 파일로 바꾸고 여기에 형태소 분석을 실시하는 방법을 설명한다. 형태소 분석 방법에는 국어학 전문가들이 태깅한 데이터로 학습된 모델로 분석하는 지도 학습 기법, 우리가 가진 말뭉치의 패텅을 학습한 모델을 적용하는 비지도 학습 기법 등이 있다.

## 3.1 데이터 확보
- 임베딩 학습용 말뭉치는 직접 만들거나, 웹 스크래핑을 하여 모을 수 있다. 이 교재에서는 이미 공개돼있는 말뭉치 데이터를 활용

- 임베딩에 자연어 의미를 함축하는 비법은 자연어의 통계적 패턴 정보를 통째로 임베딩에 넣는 것이다. 자연어의 의미는 해당 언어 화자들이 실제 사용하는 일상 언어에서 드러나기 때문이다. 임베딩을 만들 때 쓰는 통계 정보는 아래 3가지 있다.

### 3.1.1 한국어 위키백과
- 한국어 위키백과 raw data를 사용하려면 도커 컨테이너 bash shell을 사용하거나 pythn 상에서 코드로 불러 올 수 있다.

코드 3-1 bash 한국어 위키백과 다운로드
- git pull origin master
- bash preprocess.sh dump-raw-wiki



In [ ]:
cd '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data'

/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data


In [ ]:
# 코드 3-1 한국어 위키백과 다운로드
!pip install wikiextractor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 KB 3.0 MB/s eta 0:00:00


In [ ]:
# !wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2
# kowiki-latest-pages-articles.xml.bz2.1이 다운되는 현상이 발생하여 kowiki-latest-pages-articles.xml.bz2를 직접적으로 다운로드하여 사용

--2023-03-18 06:45:10--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889750893 (849M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-articles.xml.bz2.1’

kowiki-latest-pages 100%[===================>] 848.53M  4.17MB/s    in 3m 22s  

2023-03-18 06:48:32 (4.21 MB/s) - ‘kowiki-latest-pages-articles.xml.bz2.1’ saved [889750893/889750893]



In [ ]:
# 위키익스트랙터를 사용하여 위키피디아 덤프를 파싱
!python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2

In [ ]:
ls

images/                                    LICENSE
install_mecab-ko_on_colab190912.sh         README.md
install_mecab-ko_on_colab_light_220429.sh  text/
kowiki-latest-pages-articles.xml.bz2


In [ ]:
# 코드 3-2 한국어 위키백과 전처리
import os
import re

os.listdir('text')

['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI']

In [ ]:
# 한국어 위키백과 전처리

from gensim.corpora import WikiCorpus, Dictionary
from gensim.utils import to_unicode
from gensim.corpora.wikicorpus import tokenize

in_f = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/kowiki-latest-pages-articles.xml.bz2'
out_f = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/precessed_wiki_ko.txt'
output = open(out_f, 'w')
wiki = WikiCorpus(in_f, tokenizer_func = tokenize, dictionary=Dictionary())
i = 0
for text in wiki.get_texts():
    output.write(bytes(' '.join(text), 'utf-8').decode('utf-8') + '\n')
    i = i + 1
    if (i % 10000 == 0):
        print('Processed ' + str(i) + ' article')
output.close()
print('Processing complete!')

- 본문 <text> 태그 안에서 필요한 내용을 추출하는 과정이 필요하다. 여기서 특수문자, 목차, 이메일 주소 등 불필요한 문자열 제거가 필요하다.

- 코드 3-2에서 tokenize 함수를 코드 3-4와 같이 수정하여 불 필요한 정보를 제거할 것이다.

In [ ]:
# 코드 3-4 사용자 정의, 한국어 위키 토크나이저

import re
from gensim.utils import to_unicode

WIKI_REMOVE_CHARS = re.compile("'+|(=+.{2,30})=+|__TOC__|(ファイル:).+|:(en|de|it|fr|es|kr|zh|no|fi):|\n", re.UNICODE)
WIKI_SPACE_CHARS = re.compile("(\\s|゙|゚|　)+", re.UNICODE)
EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
URL_PATTENR = re.compile("((ftp|http|https):\/\/)?(www.)?(?!.*(ftp|http|https|www.))[a-zA-Z0-9_-]+(\.[a-zA-Z]+)+((\/)[\w#]+)*(\/\/[a-zA-Z0-9_]+=\w+(&[a-zA-Z0-9_]+=\w+)?$)", re.UNICODE)
WIKI_REMOVE_TOKEN_CHARS = re.compile("(\\*$|:$|^파일:.+|^;)", re.UNICODE)
MULTILPE_SPACES = re.compile(' +', re.UNICODE)
def tokenizer(content, token_min_len = 2, token_max_len = 100, lower=True):
    content = re.sub(EMAIL_PATTERN, ' ', content)
    content = re.sub(URL_PATTENR, ' ', content)
    content = re.sub(WIKI_REMOVE_CHARS, ' ', content)
    content = re.sub(WIKI_SPACE_CHARS, ' ', content)
    content = re.sub(MULTILPE_SPACES, ' ', content)
    tokens = content.replace(", )", "").split(" ")
    result = []
    for token in tokens:
        if not token.startswith('_'):
            token_candidate = to_unicode(re.sub(WIKI_REMOVE_TOKEN_CHARS, '', token))
        else:
            token_candidate = ""
        if len(token_candidate) > 0:
            result.append(token_candidate)
    return result

In [ ]:
# 한국어 위키백과 최종 전처리

content = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/precessed_wiki_ko.txt'
tokenizer(content, token_min_len = 2, token_max_len = 100, lower=True)

['/content/drive/MyDrive/Book/Sentence',
 'Embedding',
 'Using',
 'Korean',
 'Corpora/data/Mecab-ko-for-Google-Colab/precessed_wiki_ko.txt']

### 3.1.2 KorQuAD
- KorQuAD는 한국어 기계 독해를 위한 데이터셋이다. LG CNS가 구축해 2018년 공개했으며 학습/데브/테스트셋을 모두 포함해 7만 79건에 이르는 데이터이다.

- KorQuAD는 구축 전 과정에서 사람들이 개입하여 검증이 철저하게 이루어졌다.

In [ ]:
# KorQuAD 데이터 셋 다운로드

!wget https://raw.githubusercontent.com/nate-parrott/squad/master/data/train-v1.1.json
!wget https://raw.githubusercontent.com/nate-parrott/squad/master/data/dev-v1.1.json

--2023-03-20 04:17:36--  https://raw.githubusercontent.com/nate-parrott/squad/master/data/train-v1.1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [text/plain]
Saving to: ‘train-v1.1.json’

train-v1.1.json     100%[===================>]  28.88M  64.8MB/s    in 0.4s    

2023-03-20 04:17:40 (64.8 MB/s) - ‘train-v1.1.json’ saved [30288272/30288272]

--2023-03-20 04:17:40--  https://raw.githubusercontent.com/nate-parrott/squad/master/data/dev-v1.1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6

In [ ]:
# 코드 3-8 KorQuAD 전처리 (학습용)

import json

corpus_fname = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/KorQuAD_v1.0_train.json'
output_fname = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/precessed_korquad_train.txt'

with open(corpus_fname) as f1, open(output_fname, 'w', encoding = 'utf-8') as f2:
    dataset_json = json.load(f1)
    dataset = dataset_json['data']
    for article in dataset:
        w_lines = []
        for paragraph in article['paragraphs']:
            w_lines.append(paragraph['context'])
            for qa in paragraph['qas']:
                q_text = qa['question']
                for a in qa['answers']:
                    a_text = a['text']
                    w_lines.append(q_text + " " + a_text)
        for line in w_lines:
            f2.writelines(line + "\n")

In [ ]:
# 코드 3-8 KorQuAD 전처리 (데브용)

import json

corpus_fname = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/KorQuAD_v1.0_dev.json'
output_fname = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/precessed_korquad_dev.txt'

with open(corpus_fname) as f1, open(output_fname, 'w', encoding = 'utf-8') as f2:
    dataset_json = json.load(f1)
    dataset = dataset_json['data']
    for article in dataset:
        w_lines = []
        for paragraph in article['paragraphs']:
            w_lines.append(paragraph['context'])
            for qa in paragraph['qas']:
                q_text = qa['question']
                for a in qa['answers']:
                    a_text = a['text']
                    w_lines.append(q_text + " " + a_text)
        for line in w_lines:
            f2.writelines(line + "\n")

### 3.1. 네이버 영화 리뷰 말뭉치
- 네이버 영화 리뷰 말뭉치는 영화 리뷰들을 평점과 함께 수록한 한국어 말뭉치이다. 감성 분석이나 문서 분류 태스크 수행에 제격이다. 레코드 하나는 문서(리뷰)에 대응한다. 문서 ID, 문서 내용, 레이블(긍정1, 부정0)로 구성돼 있으며 각 열은 탭 문자로 구분돼있다.

In [ ]:
# 네이버 영화 리뷰 말뭉치 다운로드

# 필요한 라이브러리 및 데이터 불러오기
import pandas as pd
#import konlpy
import re
import matplotlib.pyplot as plt
import urllib.request
import numpy as np

%matplotlib inline

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
# df = pd.read_table('ratings.txt')
# df.head()

('ratings.txt', <http.client.HTTPMessage at 0x7fd6fbba0a30>)

In [ ]:
# 코드 3-11 네이버 영화 리뷰 전처리 코드

corpus_path = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/ratings.txt'
output_fname = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/process_ratings.txt'
with_label = False

with open(corpus_path, 'r', encoding = 'utf-8') as f1, open(output_fname, 'w', encoding = 'utf-8') as f2:
    next(f1) # 헤드라인 스킵
    for line in f1:
        _, sentence, label = line.strip().split('\t')
        if not sentence: continue
        if with_label:
            f2.writelines(sentence + '\u241E' + label + '\n')
        else:
            f2.writelines(sentence + '\n')

## 3.2 지도 학습 기반 형태소 분석
- 품질 좋은 임베딩을 만들기 위해서는 문장이나 단어의 경계를 컴퓨터에 알려줘야 한다. 그렇지 않으면 어휘 집합에 속한 단어 수가 기하급수적으로 늘어나서 연산의 비효율이 발생한다. 특히 한국어는 조사와 어미가 발달한 교착어이기 때문에 이런 처리를 더 꼼꼼하게 해야한다.

- 예를 들어 동사 '가다'는 '가겠다', '가더라'라는 활용이 가능하다. 이런 활용형을 모두 어휘 집합에 넣을 수 있지만 새로운 활용형이 발생할 때마다 계속 늘려야 한다는 단점이 존재한다. 이런 문제를 해소하고자 형태소 분석 기법을 사용한다.

- 오픈소스인 Mecab을 사용하여 형태소 분석기로 사용할 수 있다. Mecab을 사용하용 활용형 어휘를 분석하면 아래와 같다.

- 가겠다 > 가, 겠, 다
- 가더라 > 가, 더라

- 이런 형태소 분석 덕분에 어휘 집합 추가 없이 처리할 수 있는 것이다.

- 교착어인 한국어는 한정된 종류의 조사와 어미를 자주 이용하기 때문에 각각에 대응하는 명사, 용언(형용사, 동사) 어간만 어휘 집합에 추가하면 취급 단어 개수를 꽤 줄일 수 있다. 형태소 분석기를 잘 사용하면 NLP의 효율성을 높일 수 있다는 것이다.

- 이 파트에서는 지도 학습 기반의 형태소 분석을 수행한다. 지도 학습이란 정답이 주어진 데이터를 모델에 넣어 패턴을 학습하게하는 방법이다. 또한 이 파트에서 설명되는 형태소 분석기들은 언어학 전문가들이 태깅한 형태소 분석 말뭉치로부터 학습된 지도 학습 기반 모델들이다.

- 태깅이란 아래처럼 모델 입력과 출력 쌍을 만드는 작업을 가리킨다. 이 모델들은 문자열이 주어질 때 사람이 알려준 정답 패턴에 최대한 가깝게 토크나이즈한다.

- 입력 : 아버지가방에들어가신다
- 출력 : 아버지, 가, 방, 에, 들어가, 신다.

### 3.2.1 KoNLPy 사용법
- KoNLPy는 Mecab, 꼬꼬마, 한나눔, Okt, 코모란 등 5개 오픈소스 형태소 분석기를 파이썬 화녕에서 사용할 수 있도록한다.

In [ ]:
# KoNLPy 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
# 코드 3-14 Mecab 활용

from konlpy.tag import Mecab
tokenizer = Mecab()
tokenizer.morphs('아버지가방에들어가신다')

['아버지', '가', '방', '에', '들어가', '신다']

In [ ]:
# 코드 3-15 Mecab 품사 정보 확인
tokenizer.pos('아버지가방에들어가신다')

[('아버지', 'NNG'),
 ('가', 'JKS'),
 ('방', 'NNG'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('신다', 'EP+EC')]

In [ ]:
# 코드 3-16 get_tokenizer 함수 만들기
# 5개 토크나이저 비교를 위함

from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

def get_tokenizer(tokenizer_name):
    if tokenizer_name == 'komoran':
        tokenizer = Komoran()
    elif tokenizer_name == 'okt':
        tokenizer = Okt()
    elif tokenizer_name == 'mecab':
        tokenizer = Mecab()
    elif tokenizer_name == 'hannanum':
        tokenizer = Hannanum()
    elif tokenizer_name == 'kkma':
        tokenizer = Kkma()
    else:
        tokenizer = Mecab()
    return tokenizer

# 코모란 사용 예시
tokenizer = get_tokenizer('komoran')
print(tokenizer.morphs('아버지가방에들어가신다'))
print(tokenizer.pos('아버지가방에들어가신다'))

['아버지', '가방', '에', '들어가', '시', 'ㄴ다']
[('아버지', 'NNG'), ('가방', 'NNP'), ('에', 'JKB'), ('들어가', 'VV'), ('시', 'EP'), ('ㄴ다', 'EC')]


In [ ]:
# Okt 사용 결과
tokenizer = get_tokenizer('okt')
print(tokenizer.morphs('아버지가방에들어가신다'))
print(tokenizer.pos('아버지가방에들어가신다'))

['아버지', '가방', '에', '들어가신다']
[('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가신다', 'Verb')]


In [ ]:
# 코모란 사용 예시
tokenizer = get_tokenizer('komoran')
tokenizer.morphs('아버지가방에들어가신다')
tokenizer.pos('아버지가방에들어가신다')

### 3.2.2 KoNLPy 내 분석기별 성능 차이 분석

![](https://konlpy-ko.readthedocs.io/ko/v0.4.3/_images/time.png)
<center>문자 개수 대비 실행 시간</center>

- Mecab이 다른 분석기 대비 상대적으로 속도가 빠르다는 것을 볼 수 있다.

### 3.2.3 Khaiii 사용법 (설치문제로 인하여 실습 보류)
- Khaiii는 카카오가 2018년 말 공개한 오픈소스 한국어 형태소 분석기이다. 국립국어원이 구축한 세종 코퍼스를 이용해 CNN모델을 적용해 학습했다. Khaiii의 아키텍처 개요는 아래 그림과 같다.

![](https://t1.daumcdn.net/thumb/R1280x0/?fname=http://t1.daumcdn.net/brunch/service/user/1oU7/image/DXlTnCNYfeYzWIR4kN428VouYKQ.png)
<center>Khaiii 아케텍처</center>

- 입력 문장을 문자 단위로 읽고 컨볼루션 필터가 이 문자들을 나눠가면서 정보를 추출한다. 이 네트워크의 말단 레이어에서는 이렇게 모은 정보들을 종합해 형태소의 경계와 품사 태그를 예측한다. GPU없이도 형태소 분석이 가능하며 실행 속도도 빠르다고 한다.

### 3.2.4 Mccab에 사용자 사전 추가하기
- 형태소 분석기에서 신경 써야할 부분은 토큰들을 어떻게 처리해야 할지다. 예를 들어 '가우스전자'라는 단어에 Mecab을 사용한다고 해보자.

- 가우스전자라는 단어가 '가우스', '전자'로 나눠질 것이다. '가우스전자' 하나로 분석됐을 때보다 데이터 분석이나 임베딩 품질이 떨어질 가능서이 발생한다. 따라서 관심 단어들을 가전에 추가해 '가우스전자'같은 단어가 하나의 토큰으로 분석될 수 있도록 강제해야한다.

- 다만 Mecab을 colab에서 사용하는 경우 새로운 세션은 지속적으로 설치해야하므로 로컬 환경에서 사용하는 것을 권장한다.

[Colab 환경에서 Mecab 사용자 사전 추가 방법](https://somjang.tistory.com/entry/Google-Colab%EC%97%90%EC%84%9C-mecab-ko-dic-%EC%82%AC%EC%9A%A9%EC%9E%90-%EC%82%AC%EC%A0%84-%EC%B6%94%EA%B0%80%ED%95%98%EA%B8%B0)

## 3.3 비지도 학습 기반 형태소 분석
- 이 파트는 비지도 학습 기반의 형태소 분석을 설명한다. 3.2와 달리 비지도 학습 기법들은 데이터의 패턴을 모델 스스로 학습하게 하여 형태소를 분석하는 방법이다. 데이터에 자주 등장하는 단어들을 형태소로 인식한다.

### 3.3.1 soynlp 형태소 분석기
- soynlp(https://github.com/lovit/soynlp)는 형태소 분석, 품사 판별 등을 지원하는 파이썬 기반 한국어 NLP 패키지이다. 비지도 학습 접근법을 지향하기 때문에 하나의 문장 혹은 문서에서보다는 어느 정도 규모가 있으면서 동질적인 문서 집합에서 잘 작동한다.

- soynlp 패키지에 포함된 형태소 분석기는 데이터의 통계량을 확인해 만든 단어 점수 표로 작동한다. 단어 점수는 크게 응집 확률과 브랜칭 엔트로피를 활용한다. 구체적으로 설명하면 주어진 문자열이 유기적으로 연결돼 자주 나타나고(응집 확률이 높고), 그 단어 앞뒤로 다양한 조사, 어미 혹은 다은 단어가 등장하는 경우(브랜칭 엔트로피가 높을 때) 해당 문자열을 형태소로 취급한다.

- 예를들어 말뭉치에서 '꿀잼'이라는 단어가 연결돼 자주 나타났다면 '꿀잼'을 형태소라고 보는 것이다(응집 확률이 높다). 한편 꿀잼 앞에 '정말', '너무' 등 문자열이, 뒤에 'ㅋㅋ', 'ㅎㅎ' 등 패턴이 다영하게 나타놨다면 이 역시 '꿀잼'을 형태소로 취급한다(브랜칭 엔트로피가 높다).

- soynlp의 형태소 분석기는 우리가 가지고 있는 말뭉치의 통계량을 바탕으로 하기 때문에 별도의 학습 과정이 필요하다. 말뭉치의 분포가 어떻게 돼 있는지 확인하고 단어별로 응집 확률과 브랜칭 엔트로피 점수표를 만드는 절차가 필요하다.

- soynlp의 단어 점수를 학습하는 코드는 3-22와 같다.

In [6]:
# soynlp 설치 
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 KB 7.6 MB/s eta 0:00:00


In [ ]:
# 코드 3-22 soynlp 단어 점수 학습

# 라이브러리 불러오기
from soynlp.word import WordExtractor

# 경로 지정
corpus_fname = '/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/process_ratings.txt'
model_fname = '/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/soyword.model'

# 점수 학습
sentences = [sent.strip() for sent in open(corpus_fname, 'r').readlines()] # 입력 문장의 문자 단위로 슬라이싱
word_extractor = WordExtractor(min_frequency = 100,
                               min_cohesion_forward = 0.05,
                               min_right_branching_entropy = 0.0)
word_extractor.train(sentences)
word_extractor.save(model_fname)

training was done. used memory 0.816 Gb


In [ ]:
# 코드 3-23 soynlp 형태소 분석

# 라이브러리 불러오기
import math
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

# 학습한 soynlp 모델 불러오기
model_fname = '/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/soyword.model'

word_extractor = WordExtractor(min_frequency = 100,
                               min_cohesion_forward = 0.05,
                               min_right_branching_entropy = 0.0)

word_extractor.load(model_fname)

scores = word_extractor.word_scores()
scores = {key:(scores[key].cohesion_forward * math.exp(scores[key].right_branching_entropy)) for key in scores.keys()}
tokenizer = LTokenizer(scores = scores)

# 토큰 생성
tokens = tokenizer.tokenize("애비는 종이었다")
print(tokens)

all cohesion probabilities was computed. # words = 6130
all branching entropies was computed # words = 123575
all accessor variety was computed # words = 123575
['애비는', '종이었다']


### 3.3.2 Google Sentencepice
- Sentencepiece는 구글에서 공개한 비지도 학습 기반 형태소 분석 패키지이다. BPE 기법 등을 지원하며 간단하게 python에 설치하여 사용이 가능하다.

- BPE(Bite Pair Encoding) 기본 원리는 말뭉치에서 가장 많이 등장하는 문자열을 병합해 문자열을 압축하는 방법이다. 아래 예시를 보자.

- aaabdaaabac라는 문자열이 있다. 여기서 'aa' 가장 많이 나타나도 이를 'z'로 치환하면 zabdzabac로 압축할 수 있다. 이 문자열에서 'ab'가 많이 나오니 'y'로 한번 더 압축하면 zydzyac가 될 수 있다.

- NLP에서 BPE가 처음 쓰인 태스크는 기계 번역(Machine Translation)이다. BPE를 활용해 토크나이즈하는 방법론의 핵심은 이렇다.

- 우선 원하는 어휘 집합의 크기가 될 때까지 반복적으로 고빈도 문자열을 병합해 어휘 집합에 추가한다. 이 방법이 BPE 학습이다.

- 학습이 끝난 후 문장 내 각 어절(띄어쓰기로 나눈 것)에 어휘 집합에 있는 서브워드가 포함돼 있을 경우 해당 서브워드를 어절에서 분리한다. 이후 어절의 나머지에서 어휘 집합에 있는 서브워드를 다시 찾고, 또 분리한다. 어절 끝까지 찾았지만 어휘 집합에 없다면 UNK으로 취급한다.

- 예를 들어 BPE를 학습한 결과 고빈도 서브워드가 '학교', '법', '먹었' 등이라고 해보자. 그러면 아래 문장은 다음과 같이 분석된다. _로 시작하는 토큰은 해당 토큰이 어절의 시작임을 나타내는 구분자다.

- 학교에서 밥을 먹었다 > _학교, 에서, _밥, 을, _먹었, 다

- 다음은 전처리를 수행한 한국어 위키백과 데이터를 사용하여 BPE 방법으로 어휘 집합을 만들어본다.

In [1]:
# SentencePiece 설치
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00


In [3]:
cd '/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data'

/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data


In [ ]:
# 코드 3-25 BPE 학습
import sentencepiece as spm

train = """--input = /content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/precessed_wiki_ko.txt \
           --model_prefix = sentpiece \
           --vocab_size = 32000 \
           --model_type = bpe --character_coverage = 0.9995"""

spm.SentencePieceTrainer.Train(train)

In [1]:
# 코드 3-25 BPE 학습
import sentencepiece as spm

corpus = "/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/precessed_wiki_ko.txt"
prefix = "sentpiece"
vocab_size = 32000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe")

In [4]:
# !mv /content/sentpiece.model /content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data

In [5]:
# !mv /content/sentpiece.vocab /content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data

- BPE로 학습한 어휘 집합을 BERT 모델에도 사용할 수 있다. BPE는 문자열 기반의 비지도 학습 기법이기 때문에 데이터만 확보할 수 있다면 어떤 언어에든 적용이 가능하다.

- Sentencepipec 학습 결과를 BERT에 사용하려면 후처리가 필요하다. '_'를 BERT에 맞게 바꾸고 [PAD], [UNK], [CLS], [MASK], [SEP] 등 스페셜 토큰을 추가한다.

- 

### 3.3.3 띄어쓰기 교정
- soynlp에는 띄어쓰기 교정도 제공한다. 이 기능은 말뭉치에서 띄어쓰기 패턴을 학습한 뒤 해당 패턴대로 교정을 수행한다. 예를 들어 학습 데이터에서 '하자고' 문자열 앞뒤로 공백이 다수 발견됐다면 예측 단계에서 '하자고'가 등장했을 때 앞뒤를 띄어서 교정하는 방식이다.

- soynlp의 띄어쓰기 교정 모델 역시 데이터의 통계량을 확인해야 하기 때문에 교정을 수행하기 전 학습이 필요하다.

- soynlp 형태소 분석이나 BPE 방식의 토크나이즈 기법은 띄어쓰기에 따라 분석 결과가 크게 달라진다. 따라서 이들 모델을 학습하기 전 띄어쓰기 교정을 먼저 적용하면 그 품석 품질이 개선될 수 있다.

In [8]:
!pip install soyspacing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
# 코드 3-28 soynlp 띄어쓰기 모듈 학습
# soyspacing 설치하고 다음 코드를 실행하라

from soyspacing.countbase import CountSpace

corpus_fname = '/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/process_ratings.txt'
model_fname = '/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/spcae-correct.model'

model = CountSpace()
model.train(corpus_fname)
model.save_model(model_fname, json_format = False)

all tags length = 1166880 --> 165338, (num_doc = 199991)

In [10]:
# 코드 3-9 soynlp 띄어쓰기 교정
from soyspacing.countbase import CountSpace

model_fname = '/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/spcae-correct.model'
model = CountSpace()
model.load_model(model_fname, json_format = False)
model.correct("어릴때보고 지금다시봐도 재밌어요")

# 띄어쓰기 교정 결과 : 어릴때 보고 지금 다시봐도 재밌어요

('어릴때 보고 지금 다시봐도 재밌어요', [0, 0, 1, 0, 1, 0, 1, 0, None, 0, 1, 0, 0, 0, 1])

## 3.4 요약
- 임베딩 학습용 말뭉치는 라인 하나가 문서면 좋다. 한국어 위키 백과, KorQuAD, 네이버 영화 말뭉치 등을 이같이 전처리했다.
- 지도 학습 기반의 형태소 분석 모델은 언어학 전문가들이 태깅한 형태소 분석 말뭉치로부터 학습된 기법이다. 이 모델들은 문자열이 주어질 때 사람이 정한 정답 패턴에 가깝게 토크나이즈한다. KoNLPy, Khaiii 등이 이 부류에 속한다.
- 비지도 학습 기반의 형태소 분석 모델은 데이터의 패턴을 모델 스스로 학습하게 함으로써 형태소를 나누는 기법이다. 데이터에 자주 등장하는 단어들을 형태소로 인식한다. soynlp, Google SentencePicec 등이 이 부류에 속한다.